In [1]:
import matplotlib.pyplot as plt
from utils import NuScenesDataSet
from models import ultra_fast_bev, ultra_fast_bev_nmsbf
import numpy as np
import tensorflow as tf 

dataroot='../../../../datasets/nuscenes/'
nuscd = NuScenesDataSet(dataroot)

2024-02-27 19:39:25.334574: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 19:39:25.334714: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 19:39:25.344082: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 19:39:25.380911: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-27 19:39:26.213351: W tensorflow/compiler/tf2

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 31.056 seconds.
Reverse indexing ...
Done reverse indexing in 5.7 seconds.


# Create data pipeline

In [2]:
batch_size = 16 #to measure inference speed we need a big batch to take advantage of compute of the gpu
train_camera_files_paths, val_camera_files_paths = nuscd.get_train_val_cameras_file_paths()

file_path_lidar_train = '../../../../datasets/lidar_voxels_train/'
file_path_lidar_val = '../../../../datasets/lidar_voxels_val/'
train_lidar_files_paths, val_lidar_files_paths = nuscd.get_train_val_file_paths(file_path_lidar_train, file_path_lidar_val)

file_path_bev_mask_train = '../../../../datasets/nusc_bev_mask_train/'
file_path_bev_mask_val = '../../../../datasets/nusc_bev_mask_val/'
train__bev_mask_files_paths, val__bev_mask_files_paths = nuscd.get_train_val_file_paths(file_path_bev_mask_train, file_path_bev_mask_val)

# train_camera_files_paths = np.array(train_camera_files_paths)
# train_lidar_files_paths = np.array(train_lidar_files_paths)[..., np.newaxis]
# train__bev_mask_files_paths = np.array(train__bev_mask_files_paths)[..., np.newaxis]
val_camera_files_paths = np.array(val_camera_files_paths)
val_lidar_files_paths = np.array(val_lidar_files_paths)[..., np.newaxis]
val__bev_mask_files_paths = np.array(val__bev_mask_files_paths)[..., np.newaxis]

# train_files_paths = np.concatenate((train_camera_files_paths, train_lidar_files_paths, train__bev_mask_files_paths), axis=-1)
val_files_paths = np.concatenate((val_camera_files_paths, val_lidar_files_paths, val__bev_mask_files_paths), axis=-1)

val_index_day, val_index_night = nuscd.get_day_night_val_samples_indexes()
val_files_paths_day = val_files_paths[val_index_day]
val_files_paths_night = val_files_paths[val_index_night]

def get_image(image_path):
    image_shape = (448, 800)
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_shape)
    return image

def parse_samples(file_paths):
    image0 = get_image(file_paths[0])
    image1 = get_image(file_paths[1])
    image2 = get_image(file_paths[2])
    image3 = get_image(file_paths[3])
    image4 = get_image(file_paths[4])
    image5 = get_image(file_paths[5])

    lidar = nuscd.read_binary_file(file_paths[6], dtype=tf.float32, shape=(200,200,4))
    bev_mask = nuscd.read_binary_file(file_paths[7], dtype=tf.float32, shape=(200,200,3))
    
    return ((image0, image1, image2, image3, image4, image5, lidar), bev_mask)

# train_dataset = tf.data.Dataset.from_tensor_slices(train_files_paths)
val_dataset = tf.data.Dataset.from_tensor_slices(val_files_paths)
val_dataset_day = tf.data.Dataset.from_tensor_slices(val_files_paths_day)
val_dataset_night = tf.data.Dataset.from_tensor_slices(val_files_paths_night)

# train_dataset = train_dataset.shuffle(buffer_size=train_dataset.cardinality(), reshuffle_each_iteration=True).map(parse_samples, num_parallel_calls=tf.data.AUTOTUNE)
# train_dataset = train_dataset.batch(batch_size).prefetch(1)

val_dataset = val_dataset.map(parse_samples, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset= val_dataset.batch(batch_size).prefetch(1)

val_dataset_day = val_dataset_day.map(parse_samples, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset_day= val_dataset_day.batch(batch_size).prefetch(1)

val_dataset_night = val_dataset_night.map(parse_samples, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset_night= val_dataset_night.batch(batch_size).prefetch(1)

print(val_files_paths.shape[0], val_files_paths_day.shape[0], val_files_paths_night.shape[0])
print(val_dataset.cardinality().numpy(), val_dataset_day.cardinality().numpy(), val_dataset_night.cardinality().numpy())

2024-02-27 19:40:04.382131: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0d:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-27 19:40:04.522355: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0d:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-27 19:40:04.522419: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0d:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-27 19:40:04.524696: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0d:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-27 19:40:04.524742: I external/local_xla/xla/stream_executor

6019 5417 602
377 339 38


# Load model

In [3]:
tf.config.optimizer.set_jit(True) # Enable XLA
tf.keras.mixed_precision.set_global_policy('mixed_float16')
cameras_LUTs1 = nuscd.create_images_LUTs(image_size=(56,100))
cameras_LUTs2 = nuscd.create_images_LUTs(image_size=(112,200))
cameras_LUTs3 = nuscd.create_images_LUTs(image_size=(224,400))
cameras_LUTs = [cameras_LUTs1, cameras_LUTs1, cameras_LUTs2, cameras_LUTs3] 

#Multy scale BEV features
# model = ultra_fast_bev((448,800,3), cameras_LUTs, 3, 'EfficientNetB1',  'GroupNormalization', './model_weights/image_encoder_effnetb1_groupnorm_bs20_imagenet.h5')
# model.load_weights('./model_weights/ufb_groupnorm_bs2.h5')

# model = ultra_fast_bev((448,800,3), cameras_LUTs, 3, 'EfficientNetB1',  'BatchNormalization', './model_weights/image_encoder_effnetb1_batchnorm_bs20_imagenet.h5')
# model.load_weights('./model_weights/ufb_batchnorm_bs2.h5')

# model = ultra_fast_bev((448,800,3), cameras_LUTs, 3, 'EfficientNetB1',  'BatchNormalization', './model_weights/image_encoder_effnetb1_batchnorm_bs20_imagenet.h5', 'C')
# model.load_weights('./model_weights/ufb_batchnorm_bs4_nl.h5')

#No Multy scale BEV features
# model = ultra_fast_bev_nmsbf((448,800,3), cameras_LUTs[0:1], 3, 'EfficientNetB1',  'GroupNormalization', './model_weights/image_encoder_effnetb1_groupnorm_bs20_imagenet.h5')
# model.load_weights('./model_weights/ufb_groupnorm_bs2_nmsbf.h5')

model = ultra_fast_bev_nmsbf((448,800,3), cameras_LUTs[0:1], 3, 'EfficientNetB1',  'BatchNormalization', './model_weights/image_encoder_effnetb1_batchnorm_bs20_imagenet.h5')
# model.load_weights('./model_weights/ufb_batchnorm_bs2_nmsbf.h5')
# model.load_weights('./model_weights/ufb_batchnorm_bs4_nmsbf.h5')
# model.load_weights('./model_weights/ufb_batchnorm_bs6_nmsbf.h5')
model.load_weights('./model_weights/ufb_batchnorm_bs5_nmsbf_run2.h5')

# model = ultra_fast_bev_nmsbf((448,800,3), cameras_LUTs[0:1], 3, 'EfficientNetB1',  'BatchNormalization', './model_weights/image_encoder_effnetb1_batchnorm_bs20_imagenet.h5', 'C')
# model.load_weights('./model_weights/ufb_batchnorm_bs4_nmsbf_nl.h5')


metrics = [
    tf.keras.metrics.MeanIoU(num_classes=3, name = "Mean_IOU", sparse_y_true= False, sparse_y_pred=False),
    tf.keras.metrics.IoU(num_classes=3, name = "road", target_class_ids = [0], sparse_y_true= False, sparse_y_pred=False),
    tf.keras.metrics.IoU(num_classes=3, name = "car", target_class_ids = [1], sparse_y_true= False, sparse_y_pred=False),
    tf.keras.metrics.IoU(num_classes=3, name = "nothing", target_class_ids = [2], sparse_y_true= False, sparse_y_pred=False)
    ]

model.compile(metrics=metrics)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6


2024-02-27 19:40:35.511293: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0d:00.0/numa_node
Your kernel may have been built without NUMA support.


# Wrong Evaluation

In [16]:
#importan! for some reason model.evaluate dosent evaluate right our model for the classes road and nothing when we use mixed_float16
#if we split the data into day and night and calculate the weighted avarage mean_iou it should give as the iou of the hole dataset which it dosent
#there seems to be a acumilation error that increases the biger the dataset is 
#if you want to evaluate using mixed presision use the method below, if you are going to use float32 use this method 

# model.evaluate(val_dataset)
# model.evaluate(val_dataset_day) 
# model.evaluate(val_dataset_night) 

#Multy scale BEV features, wsl (ubuntu 22.04)
#groupnorm_bs2 
#inference time per sample: 41ms
# Mean_IOU: 0.5760 - road: 0.6070 - car: 0.5051 - nothing: 0.6160 
# Mean_IOU: 0.5872 - road: 0.6244 - car: 0.5041 - nothing: 0.6332 - day
# Mean_IOU: 0.7537 - road: 0.7637 - car: 0.5397 - nothing: 0.9577 - night

#batch_norm_bs2 
#inference time per sample: 35ms
# Mean_IOU: 0.5448 - road: 0.5688 - car: 0.4883 - nothing: 0.5774, 
# Mean_IOU: 0.5557 - road: 0.5856 - car: 0.4873 - nothing: 0.5941 - day
# Mean_IOU: 0.7378 - road: 0.7369 - car: 0.5229 - nothing: 0.9536 - night 



# Right Evaluation

In [4]:
def iou_metrics_reset():
    metrics[0].reset_state()
    metrics[1].reset_state()
    metrics[2].reset_state()
    metrics[3].reset_state()

def iou_metrics_update(y, y_pred):
    metrics[0].update_state(y, y_pred)
    metrics[1].update_state(y, y_pred)
    metrics[2].update_state(y, y_pred)
    metrics[3].update_state(y, y_pred)

def iou_metrics_results():
    res1 = metrics[0].result().numpy()
    res2 = metrics[1].result().numpy()
    res3 = metrics[2].result().numpy()
    res4 = metrics[3].result().numpy()

    return np.array([res1, res2,res3, res4])

## inference time in fps

In [5]:
@tf.function #wont compile with jit during inferece unless wraped in tf.function
def tf_model(x):
    return model(x)

In [6]:
import time
idx = 0
samples_n = 21
for x, y in val_dataset.take(samples_n):
    if idx==1: #start measuring from the second loop because model might compile in the fist loop
        time_start = time.time()
    y_pred = tf_model(x)
    idx+=1
time_end = time.time()
time_per_sample = (time_end-time_start)/(batch_size*(samples_n-1))
print(f'fps = {1/time_per_sample}')

2024-02-17 13:01:07.199208: I external/local_xla/xla/service/service.cc:168] XLA service 0x95e66ff0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-17 13:01:07.199250: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-02-17 13:01:07.270473: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-17 13:01:07.890349: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-17 13:01:08.779569: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-17 13:01:28.616626: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng4{} for conv (f16[96,112,200,96]{3,2,1,0}, u8[0]{0}) custom-call(f16[96,225,401,96]{3,2,1,0}, f16[

fps = 69.95660136300408


In [7]:
iou_metrics = np.array([0,0,0,0],dtype=np.float32)
iou_metrics_day = np.array([0,0,0,0],dtype=np.float32)
iou_metrics_night = np.array([0,0,0,0],dtype=np.float32)

iou_metrics_reset()
for x,y in val_dataset:
    y_pred = tf_model(x)
    iou_metrics_update(y, y_pred)
iou_metrics = iou_metrics_results()

iou_metrics_reset()
for x,y in val_dataset_day:
    y_pred = tf_model(x)
    iou_metrics_update(y, y_pred)
iou_metrics_day = iou_metrics_results()

iou_metrics_reset()
for x,y in val_dataset_night:
    y_pred = tf_model(x)
    iou_metrics_update(y, y_pred)
iou_metrics_night = iou_metrics_results()


print(iou_metrics)
print(iou_metrics_day)
print(iou_metrics_night)

#Evaluation metrics 
#Mean_IOU, road_iou, car_iou, nothing_iou

#Multy scale BEV features
#group norm, batchsize=2
#inference speed: 51.0 fps
# [0.7592291  0.8294191 0.5050622 0.9432061]
# [0.7591116  0.83296883 0.50410104 0.9402649 ] - day
# [0.7562861  0.7637147  0.53965276 0.9654909 ] - night

#batch norm, batchsize=2 
#inference speed: 62.9 fps
# [0.7412142  0.80142915 0.49831204 0.93390137]
# [0.7407832  0.804848   0.4873187  0.93018293] - day
# [0.7406412  0.7368739  0.5228709  0.96217865] - night

#batch norm, batchsize=4, NO Lidar
#inference speed: 62
# [0.6381374  0.7344777  0.26831377 0.91162086]
# [0.6387157  0.7402352  0.26864916 0.9072625 ]
# [0.609385   0.62727    0.2560359  0.94484913]


# No Multy scale BEV features
#group norm, batchsize=2
#inference speed: 57.4 fps
# [0.7464096  0.80847263 0.49444437 0.93631184]
# [0.7459523  0.8116656  0.4934519  0.93273926]
# [0.7473073 0.7490108 0.5294082 0.963503]

#batch norm, batchsize=2 
#inference speed: 77.9
# [0.75144154 0.80747795 0.5106551  0.9361916 ]
# [0.7509284  0.81045216 0.5098626  0.9324705 ]
# [0.75132614 0.75129515 0.5381671  0.96451616]

#batch norm, batchsize=4
#inference speed: 76.9 fps
# [0.7517066  0.80723876 0.51169354 0.9361876 ]
# [0.7512619  0.81029767 0.51095164 0.9325364 ]
# [0.7504608  0.7498395  0.53755105 0.963992 ]

#batch norm, batchsize=5 run2,  trained using weights from ufb_batchnorm_bs4_nmsbf
#inference speed: 76.9 fps
# [0.76694775 0.82793605 0.5293352  0.9435721 ]
# [0.76714224 0.8315789  0.52917093 0.9406768 ]
# [0.75359184 0.7601371  0.53510743 0.96553123]

#batch norm, batchsize=4, NO Lidar
#inference speed: 74 fps
# [0.64134747 0.7353326  0.27623364 0.9124761 ]
# [0.6418464 0.7407881 0.2767292 0.908022 ]
# [0.6123806  0.6328632  0.25783348 0.94644517]

2024-02-17 13:05:39.816826: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng4{} for conv (f16[54,112,200,96]{3,2,1,0}, u8[0]{0}) custom-call(f16[54,225,401,96]{3,2,1,0}, f16[96,3,3,1]{3,2,1,0}), window={size=3x3 stride=2x2}, dim_labels=b01f_o01i->b01f, feature_group_count=96, custom_call_target="__cudnn$convForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-02-17 13:05:41.069057: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 2.252331693s
Trying algorithm eng4{} for conv (f16[54,112,200,96]{3,2,1,0}, u8[0]{0}) custom-call(f16[54,225,401,96]{3,2,1,0}, f16[96,3,3,1]{3,2,1,0}), window={size=3x3 stride=2x2}, dim_labels=b01f_o01i->b01f, feature_group_count=96, custom_call_target="__cudnn$convForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-02-1

[0.76694775 0.82793605 0.5293352  0.9435721 ]
[0.76714224 0.8315789  0.52917093 0.9406768 ]
[0.75359184 0.7601371  0.53510743 0.96553123]


# Show results 

In [5]:
from matplotlib import pyplot as plt
from IPython.display import clear_output
import cv2 
import keyboard
import os 

stop = 1
key = ''
sample = 0
scale = 1/255.0
y_pred_fig = None
for x, y in val_dataset_night.unbatch().batch(1):
    y_pred = model(x).numpy().reshape((200,200,3))
    y_pred = tf.one_hot(np.argmax(y_pred, axis=-1),depth=3).numpy()
    y_pred = nuscd.add_ego_vehicle_to_mask(y_pred, distance_around_ego=(100,100))
    y_pred = nuscd.from_mask_to_rgb(y_pred)

    y = nuscd.add_ego_vehicle_to_mask(y.numpy().reshape((200,200,3)), distance_around_ego=(100,100))
    y = nuscd.from_mask_to_rgb(y)

    lidar = (np.sum(x[6].numpy()[0, :, :, :], axis=-1)>=1)[..., np.newaxis]
    lidar = np.concatenate((lidar,lidar,lidar),axis=-1)

    # y =  np.clip(y + lidar *0.4, 0, 1)

    top_img = tf.concat((x[1],x[0],x[2]), axis=2)
    bottom_img = tf.concat((x[5],x[3],x[4]), axis=2)
    img = tf.concat((top_img, bottom_img), axis=1)[0].numpy()*scale
    
    bev = np.concatenate((y, np.ones((y.shape[0],1,3), dtype=np.float32), y_pred), axis=1)
    bev = cv2.resize(bev, (img.shape[1], int(bev.shape[0]*img.shape[1]/bev.shape[1]) ), cv2.INTER_LINEAR)

    plot = np.concatenate((img, np.ones((6, img.shape[1],3)),bev), axis=0)

    cv2.imshow('BEV',np.flip(plot, axis=-1))
    
    if key == ord('s') or stop ==1:
        key = cv2.waitKey(0) & 0xFF
        stop = 1

    if key != ord('s') or stop==0:
        key = cv2.waitKey(1) & 0xFF
        stop = 0

    if key == ord('q'):
        break

    clear_output()
    print(f'sample {sample}')
    sample+=1
cv2.destroyAllWindows()

sample 601


In [22]:
# scale = 1/255.0
# for x ,y in val_dataset_night.take(5):
#     top_img = tf.concat((x[1],x[0],x[2]), axis=2)
#     bottom_img = tf.concat((x[5],x[3],x[4]), axis=2)
#     img = tf.concat((top_img, bottom_img), axis=1)[0].numpy()*scale
#     lidar = x[6][0].numpy()
#     bev_mask = y[0].numpy()
#     bev_mask = nuscd.add_ego_vehicle_to_mask(bev_mask)
#     bev_mask = nuscd.from_mask_to_rgb(bev_mask)

#     plt.figure(figsize=(20,20))
#     plt.subplot(2,2,(1,2))
#     plt.axis(False)
#     plt.imshow(img)

#     plt.subplot(2,2,3)
#     plt.axis(False)
#     plt.imshow(bev_mask)

#     plt.subplot(2,2,4)
#     plt.axis(False)
#     plt.imshow((np.sum(lidar, axis=-1)>=1)*1.0, cmap='gray')

#     plt.subplots_adjust(wspace=0.0, hspace=0.0) 
#     plt.show()

